In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes xformers langchain_community gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import gradio as gr
from langchain import HuggingFacePipeline, PromptTemplate, LLMChain
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
model_id = "Kishorereddy123/llama2_finetuned"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

...

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

In [ ]:
prompt_template = """
You are an helpful python AI. Help users with their queries.
Task: {question}
Answer:"""

prompt = PromptTemplate(template=prompt_template, input_variables=[ "question"])


In [ ]:
def create_pipeline(max_new_tokens=150):
    pipe = pipeline("text-generation",
                    model=model,
                    tokenizer=tokenizer,
                    max_new_tokens=max_new_tokens,
                    temperature=0.7)
    return pipe

In [ ]:
class PythonBot:
    def __init__(self, prompt,  task: str = "text-generation"):
        self.prompt = prompt


    def create_chat_bot(self, max_new_tokens=512):
        hf_pipe = create_pipeline(max_new_tokens)
        llm = HuggingFacePipeline(pipeline=hf_pipe)
        qa = LLMChain(
            llm=llm,
            prompt=self.prompt
        )
        return qa



python_bot = PythonBot(prompt=prompt)
bot = python_bot.create_chat_bot()


def respond(message, chat_history):

    response = bot.run({"question": message})
    print("Full Bot response:", response)


    if isinstance(response, dict):
        print("Response is a dictionary with keys:", response.keys())
        if 'text' in response:
            bot_message = response['text']
        elif 'answer' in response:
            bot_message = response['answer']
        else:
            bot_message = "Response keys are unexpected: " + ", ".join(response.keys())
    elif isinstance(response, str):
        bot_message = response
    else:
        bot_message = "Response format is not recognized."

    chat_history.append((message, bot_message))
    return "", chat_history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Python Bot", height=300)
    msg = gr.Textbox(label="Ask your question", placeholder="Type here...")
    submit_button = gr.Button("Submit")

    submit_button.click(
        fn=respond,
        inputs=[msg, chatbot],
        outputs=[msg, chatbot]
    )

demo.launch(share=True, debug=True)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
<ipython-input-6-d037ad6797ce>:8: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  llm = HuggingFacePipeline(pipeline=hf_pipe)
<ipython-input-6-d037ad6797ce>:9: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  qa = LLMChain(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://dfd0e10d4f2d846b29.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


<ipython-input-6-d037ad6797ce>:23: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = bot.run({"question": message})


Full Bot response: 
You are an helpful python AI. Help users with their queries.
Task: Hello!
Answer: Hello there! How may I assist you today? Please let me know if you have any questions or queries. 
Full Bot response: 
You are an helpful python AI. Help users with their queries.
Task: How does randit function work, and what parameters does it take?
Answer: The random function in Python takes no parameters and generates a random float between 0.0 and 1.0 (inclusive). The random module provides a function called random() which generates a random float between 0.0 and 1.0. Here is an example: 
random_num = random.random() 
print(random_num) 
Output: 0.3276375527336543 
Note that random.randint(low, high) generates a random integer between low and high (inclusive). 
For example: 
random_num = random.randint(1, 10) 
print(random_num) 
Output: 7 




Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://dfd0e10d4f2d846b29.gradio.live
